In [ ]:
import requests
import obi_one as obi
virtual_lab_id=obi.LAB_ID_STAGING_TEST
project_id=obi.PROJECT_ID_STAGING_TEST
from entitysdk import Client, ProjectContext

from obi_auth import get_token
from obi_notebook import get_projects

obi_one_api_url = "http://127.0.0.1:8100"

token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=obi.LAB_ID_STAGING_TEST, project_id=obi.PROJECT_ID_STAGING_TEST)
db_client = Client(api_url="https://staging.openbraininstitute.org/api/entitycore", project_context=project_context, token_manager=token)

In [ ]:
memodel_id = "82c27da0-2d11-4e40-a594-12f5519effed"

In [ ]:

# Sim duration
sim_duration = 3000.0

sim_conf = obi.MEModelSimulationScanConfig.empty_config()

# Info
info = obi.Info(campaign_name="MEModel Simulation", campaign_description="Simulation of MEModel circuit with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Regular Timesteps
regular_timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=sim_duration)
sim_conf.add(regular_timestamps, name='RegularTimestamps')

# Recordings
voltage_recording = obi.SomaVoltageRecording()
sim_conf.add(voltage_recording, name='VoltageRecording')

time_window_voltage_recording = obi.TimeWindowSomaVoltageRecording(start_time=0.0, end_time=2000.0)
sim_conf.add(time_window_voltage_recording, name='TimeWindowVoltageRecording')

# Initialization
simulations_initialize = obi.MEModelSimulationScanConfig.Initialize(circuit=obi.MEModelFromID(id_str=memodel_id), 
                                                        simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()





In [ ]:
import tempfile

# Create a temporary directory (not used in this example, but can be useful for testing)
with tempfile.TemporaryDirectory() as temp_dir:

    """
    Very strange issue with adding assets when a relative path is used here. Need to look into after CNS.
    """
    grid_scan = obi.GridScanGenerationTask(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='../../../obi-output/memodel_simulations/grid_scan')
    # grid_scan = obi.GridScanGenerationTask(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root=temp_dir)
    grid_scan.multiple_value_parameters(display=True)
    grid_scan.coordinate_parameters(display=True)
    grid_scan.execute(db_client=db_client)
    obi.run_tasks_for_generated_scan(grid_scan, db_client=db_client)